In [1]:
# Bibliotecas importadas
from datetime import datetime
from time import sleep
import pandas as pd
import requests
import random
import sqlite3
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
options = Options()
options.binary_location = r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe'
driver = webdriver.Firefox(executable_path='geckodriver.exe', options=options)

C:\Users\UFT\AppData\Local\Temp\ipykernel_7228\446415474.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='geckodriver.exe', options=options)


In [2]:
# Função de tempo de espera
def time_sleep(minimo=0.5, maximo=1.5):
    dt = minimo + random.random() * (maximo - minimo)
    sleep(dt)

In [3]:
def obtem_url_from_form_click(driver, url, btn_xp):
    try:
        driver.get(url)
        time_sleep()
        
        driver.find_element(By.XPATH, btn_xp).click()

        time_sleep()
        url = driver.current_url
        return url
    except:
        return None   

In [4]:
def get_empresas_listadas(url, driver):
    # Elementos XPath 
    btn_todos_xp = '//*[@id="accordionName"]/div/app-companies-home-filter-name/form/div/div[4]/button'
    formato_tabela_xp = '//*[@id="nav-table-tab"]'
    total_pages_xp = '//*[@id="listing_pagination"]/pagination-template/ul/li[9]/a/span[2]'
    next_page_xp = '//*[@id="listing_pagination"]/pagination-template/ul/li[10]'
    
    # Captura URL com todas as empresas listadas
    #driver = webdriver.Chrome()
    url_todos = obtem_url_from_form_click(driver, url_base, btn_todos_xp)

    # Tempo de espera para carregamento dos resultados
    time_sleep()
    
    # Altera a disposição dos elementos da página para tabular
    driver.find_element(By.XPATH, formato_tabela_xp).click()
    
    # Obtém o número de páginas
    total_pages = driver.find_element(By.XPATH, total_pages_xp).text

    # Cria um dataframe vazio
    tbl_df = pd.DataFrame()

    # Loop todas as páginas
    for page in range(int(total_pages)):
        # Tempo de espera para carregamento dos resultados
        time_sleep()

        # Obtém o código fonte de cada página e converte em um objeto Beaultifulsoup
        page = driver.page_source
        html = BeautifulSoup(page)
        
        # Captura o primeriro elemento tabela (em um lista)
        div_table = html.find('div', {'id':'nav-table'})

        # Converte a tabela para um dataframe
        table_list = pd.read_html(str(div_table))
        
        # Apensa o dataframe obtido na página ao dataframe 'tbl_df'
        tbl_df = tbl_df.append(table_list[0].fillna(''))
        
        # Clica para a próxima página
        driver.find_element(By.XPATH, next_page_xp).click()

    today = str(datetime.today()).split()[0]
    tbl_df.to_csv(f'empresas_listadas_{today}.csv', sep=';', encoding='utf-8', index=False)
    return tbl_df

In [44]:
empresas_listadas_df = get_empresas_listadas(url_base, driver)

C:\Users\UFT\AppData\Local\Temp\ipykernel_4304\3223847348.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tbl_df = tbl_df.append(table_list[0].fillna(''))
C:\Users\UFT\AppData\Local\Temp\ipykernel_4304\3223847348.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tbl_df = tbl_df.append(table_list[0].fillna(''))
C:\Users\UFT\AppData\Local\Temp\ipykernel_4304\3223847348.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tbl_df = tbl_df.append(table_list[0].fillna(''))
C:\Users\UFT\AppData\Local\Temp\ipykernel_4304\3223847348.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tbl_df = tbl_df.append(table_list[0].fillna(''))
C:\U

In [45]:
empresas_listadas_df.head()

,Razão Social,Nome de Pregão,Segmento,Código
0,3R PETROLEUM ÓLEO E GÁS S.A,3R PETROLEUM,NM,RRRP
1,3Z REALTY DESENVOLVIMENTO IMOBILIARIO S/A,3ZREALTY,,TZRD
2,524 PARTICIPACOES S.A.,524 PARTICIP,MB,QVQP
3,A GERADORA ALUGUEL DE MAQUINAS S.A.,GERADORA,,GRDR
4,A BODYTECH PARTICIPACOES S.A.,BODYTECH,,BODY


In [47]:
dict(empresas_listadas_df.iloc[378])

{'Razão Social': 'CEMIG DISTRIBUICAO S.A.',
 'Nome de Pregão': 'CEMIG DIST',
 'Segmento': '',
 'Código': 'CMGD'}

In [5]:
# ['Código', 'Denominação Social', 'Tipo', 'Qtde. Teórica', 'Part. (%)']
conn = sqlite3.connect('EmpresasB3IBOV.db')

In [6]:
cursor = conn.cursor()
try:
    cursor.execute(
        '''
            CREATE TABLE empresas_b3_ibov
            (
                den_social TEXT,
                nome_pregao TEXT,
                segmento TEXT,            
                cod_b3 TEXT,
                cnpj TEXT,
                cod_cvm TEXT,
                cod_negociacao TEXT,
                acao TEXT,
                tipo TEXT,
                qtd_teorica INTEGER,
                part_perc REAL,
                atividade_principal TEXT,
                setor_economico TEXT,
                subsetor_economico TEXT,
                segmento_economico TEXT,
                site TEXT,
                url_fonte TEXT
            )
        '''
    )
except:
    pass

In [22]:
cursor.execute('SELECT * FROM empresas_b3_ibov')
cursor.fetchall()

[]

In [8]:
def obtem_url_empresa(driver, empresa):
    # URL da página de busca por empresas
    url_base = "https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/"
    
    # Elementos XPath
    input_company_name_xp = '//*[@id="keyword"]'
    btn_busca_xp = '//*[@id="accordionName"]/div/app-companies-home-filter-name/form/div/div[3]/button'
    card_empresa_xp = '//*[@id="nav-bloco"]/div/div/div/div'
    
    try:
        driver.get(url_base)
        time_sleep()
        driver.find_element(By.XPATH, input_company_name_xp).send_keys(empresa)

        driver.find_element(By.XPATH, btn_busca_xp).click()

        time_sleep()
        driver.find_element(By.XPATH, card_empresa_xp).click()

        time_sleep()
        url_empresa = driver.current_url
    except:
        url_empresa = ''
    return url_empresa

In [9]:
url_empresa = obtem_url_empresa(driver, '3R PETROLEUM ÓLEO E GÁS S.A')
url_empresa

'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/main/25291/RRRP/overview?language=pt-br'

In [10]:
def captura_conteudo(soup, title, tag_title, tag_content):
    try:
        element = soup.find(lambda tag: tag.name == tag_title and title in tag.text)
        content = element.find_next(tag_content).text.strip().replace('\n', '')
        for i in range(1,10):
            content = content.replace(i * ' ', ' ')
    except:
        content = ''
    return content

In [11]:
def split_classificacao_setorial(class_setorial):
    setorial_list_keys = [
        'Setor Econômico',
        'Subsetor Econômico',
        'Segmento Econômico',
    ]
    setorial_list_values = [_.strip() for _ in class_setorial.split('/')]
    dic = dict(zip(setorial_list_keys, setorial_list_values))
    return dic

In [12]:
def obtem_dados_empresa(driver, empresa_dict):
    d = empresa_dict
    url_empresa = obtem_url_empresa(driver, d['Código'])
    time_sleep()
    html = driver.page_source
    time_sleep()
    soup = BeautifulSoup(html)
    cnpj = captura_conteudo(soup, 'CNPJ', 'p', 'p')
    cod_cvm = captura_conteudo(soup, 'Código CVM', 'b', 'p')
    atividade_principal = captura_conteudo(soup, 'Atividade Principal', 'p', 'p')
    class_setorial = captura_conteudo(soup, 'Classificação Setorial', 'p', 'p')
    site = captura_conteudo(soup, 'Site', 'p', 'p')
    den_social = soup.find('h2').text
    #cod_negociacao = captura_conteudo(soup, 'Código de Negociação', 'p', 'p')
    dict_setorial = split_classificacao_setorial(class_setorial)
    d.update({
        'CNPJ': cnpj,
        'Código CVM': cod_cvm,
        'Denominação Social': den_social,
        'Atividade Principal': atividade_principal
    })
    d.update(dict_setorial)
    d.update({'Site': site, 'Url Fonte': url_empresa})
    return d

In [20]:
def ehDuplicado(dict_empresa):
    conn = sqlite3.connect('EmpresasB3IBOV.db')
    cursor = conn.cursor()
    query = "SELECT * FROM empresas_b3_ibov WHERE den_social = ?"
    params = [dict_empresa['Denominação Social']]
    cursor.execute(query, params)
    return len(cursor.fetchall()) != 0

In [14]:
def insere_empresa_db(dict_empresa):
    conn = sqlite3.connect('EmpresasB3IBOV.db')
    cursor = conn.cursor()
    cursor.execute(
        '''
            INSERT INTO 
                empresas_b3
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)
        ''',
        (
            dict_empresa['Denominação Social'],
            dict_empresa['Nome de Pregão'],
            dict_empresa['Segmento'],
            dict_empresa['Código'],
            dict_empresa['CNPJ'],
            dict_empresa['Código CVM'],
            dict_empresa['Código de Negociação'],
            dict_empresa['Ação'],
            dict_empresa['Tipo'],
            dict_empresa['Qtde. Teórica'],
            dict_empresa['Part. (%)'],
            dict_empresa['Atividade Principal'],
            dict_empresa['Setor Econômico'],
            dict_empresa['Subsetor Econômico'],
            dict_empresa['Segmento Econômico'],
            dict_empresa['Site'],
            dict_empresa['Url Fonte']
        )
    )
    conn.commit()
    conn.close()

In [25]:
def obtem_dados_empresas(driver, empresas_df):
    for index, empresa in empresas_df.iterrows():
        try:
            d = dict(empresa)
            #if not ehDuplicado(d):
            dict_empresa = obtem_dados_empresa(driver, dict(empresa))
            insere_empresa_db(dict_empresa)
        except:
            pass

In [16]:
empresas_ibov = pd.read_csv('data/ibov_jan-abr_2022.csv')

In [17]:
empresas_ibov.head()

,Código,Açăo,Tipo,Qtde. Teórica,Part. (%)
0,ABEV3,AMBEV S/A,ON EDJ,4359908680,3.368
1,ALPA4,ALPARGATAS,PN N1,167963777,0.310
2,AMER3,AMERICANAS,ON NM,397635158,0.629
3,ASAI3,ASSAI,ON NM,794310097,0.516
4,AZUL4,AZUL,PN N2,327780192,0.400


In [18]:
obtem_dados_empresa(driver, dict(empresas_ibov.iloc[20]))

{'Código': 'CIEL3',
 'Açăo': 'CIELO',
 'Tipo': 'ON  EJ  NM',
 'Qtde. Teórica': 1144359228,
 'Part. (%)': 0.131,
 'CNPJ': '01.027.058/0001-91',
 'Código CVM': '21733',
 'Denominação Social': 'CIELO S.A.',
 'Atividade Principal': 'Empresa prestadora de serviços de adquirência e meios de pagamento.',
 'Setor Econômico': 'Financeiro',
 'Subsetor Econômico': 'Serviços Financeiros Diversos',
 'Segmento Econômico': 'Serviços Financeiros Diversos',
 'Site': 'www.cielo.com.br',
 'Url Fonte': 'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/main/21733/CIEL/overview?language=pt-br'}

In [26]:
obtem_dados_empresas(driver, empresas_ibov)

In [70]:
cursor.close()